In [21]:
import os
import json
import pandas
import re

def json_extract(result):
    return json.loads(re.sub(r"\`{3}(json)?", "", result))

def traductor_pro(client, text, namespace, target_language):
    os.mkdir(namespace)

    prompt1 = f"""
    Detecta el idioma del siguiente texto y devuelve una lista JSON con los primeros tres códigos del idioma 
    y el código de la bandera del país que se pueda usar para generar la bandera de flagsapi.com

    Ejemplo: de un texto se extrae la lista ponderada con el idioma y posible código de país

    TEXTO... -> [[1, "spanish", "MX"], [2, "spanish", "AR"], [3, "spanish", "ES"]]

    Los códigos corresponden al API de las banderas

    https://flagsapi.com/:country_code/:style/:size.png -> https://flagsapi.com/MX/flat/64.png

    El texto para detectar el idioma y código del país es:

    ===
    {text}
    ===

    NOTA IMPORTANTE: Devuelve sólo el JSON sin texto adicional, preámbulos o las tildels ```json
    ya que se usuará directamente en python.
    """.strip()

    response1 = client.responses.create(
        model="gpt-4o",
        input=prompt1
    )

    with open(f"{namespace}/reponse1.txt", "w") as file:
        file.write(response1.output_text)
    
    report1 = pandas.DataFrame(json_extract(response1.output_text), columns=["rank", "language", "country"])

    report1.to_csv(f"{namespace}/native_langague_rank.csv", index=False)

    url_flag = f"https://flagsapi.com/{report1.iloc[0]['country']}/flat/64.png"

    with open(f"{namespace}/url_flag.txt", "w") as file:
        file.write(url_flag)

    prompt2 = f"""

    Del siguiente texto genera una lista JSON con las 20 palabras principales y qué sean difíciles de aprender
    o no cumunes en el vocabulario de una persona que está aprendiendo el lenguaje <{report1.iloc[0]['language']}>
    y quiere saber las traducciones, las siglas no son importantes ya que tienen un significado directo,
    busca las palabras que se empleen en conversaciones y se utilicen en el texto.

    El JSON debe tener la lista con las palabras, por ejemplo:

    TEXTO... -> [palabra1, palabra2, ..., palabra20]

    El texto para extraer las 20 palabras principales es:

    ===
    {text}
    ===

    NOTA IMPORTANTE: Devuelve sólo el JSON sin texto adicional, preámbulos o las tildels ```json
    ya que se usuará directamente en python.

    """.strip()

    response2 = client.responses.create(
        model="gpt-4o",
        input=prompt2
    )

    with open(f"{namespace}/reponse2.txt", "w") as file:
        file.write(response2.output_text)

    top_words = "\n".join(json_extract(response2.output_text))

    with open(f"{namespace}/top_words.txt", "w") as file:
        file.write(top_words)

    framelist_example = """

    ---

    [
        {
            "native_word": palabra1_nativo,
            "native_meaning": significado1_nativo,
            "translated_word": palabra1_objetivo,
            "translated_meaning": significado1_objetivo,
            "translated_description": descripción1_objetivo,
            "word_type": tipo1_objetivo,
            "native_example": ejemplo1_nativo,
            "translated_example": ejemplo1_objetivo,
        }
        ...
    ]

    !!! NO DEVOLVER

    {
        "palabras": [
            {
                "native_word": palabra1_nativo,
                "native_meaning": significado1_nativo,
                "translated_word": palabra1_objetivo,
                "translated_meaning": significado1_objetivo,
                "translated_description": descripción1_objetivo,
                "word_type": tipo1_objetivo,
                "native_example": ejemplo1_nativo,
                "translated_example": ejemplo1_objetivo,
            },
            ...
        ]
    }

    DEVOLVER:

    [
        [palabra1_nativo, significado1_nativo, palabra1_objetivo, significado1_objetivo, descripción1_objetivo, tipo1_objetivo, ejemplo1_nativo, ejemplo1_objetivo],
        ...
    ]

    ---
    """

    prompt3_1 = f"""

    Dada la siguiente lista de palabras genera un JSON compatible con PANDAS de PYTHON 
    con las filas para cada palabra con:

    * La palabra en su idioma natural (<{report1.iloc[0]['language']}>)
    * El significado corto en su idioma natural (<{report1.iloc[0]['language']}>)
    * La palabra en el idioma objetivo (<{target_language}>)
    * El significado corto en su idioma objetivo (<{target_language}>)
    * Descripción breve de la tradicción en el idioma objetivo (<{target_language}>)
    * Una breve descripción del tipo de palabra si es verbo, sustantivo, etc. en el idioma objetivo (<{target_language}>)
    * Un ejemplo de uso en el idioma natural (<{report1.iloc[0]['language']}>)
    * Un ejemplo de uso en el idioma objetivo (<{target_language}>)

    Ejemplo:

    Toma las PALABRAS...

    Luego, para cada palabra genera una datalist:
    
    {framelist_example}

    Las palabras a convertir en un datalist con los significados son:

    ===
    {top_words}
    ===

    NOTA IMPORTANTE: Devuelve sólo el JSON sin texto adicional, preámbulos o las tildels ```json
    ya que se usuará directamente en python.

    """.strip()

    response3_1 = client.responses.create(
        model="gpt-4o",
        input=prompt3_1
    )

    with open(f"{namespace}/reponse3_1.txt", "w") as file:
        file.write(response3_1.output_text)

    pandas.DataFrame(
        json_extract(response3_1.output_text),
        columns=["native_word", "native_meaning", "translated_word", "translated_meaning", "translated_description", "type", "native_example", "transalate_example"]
    ).to_csv(f"{namespace}/table_meaning.csv", index=False)

    prompt4_1 = f"""

    Realiza la traducción lo más fiel posible e indica con etiquetas <abbr> el significado de palabras no comunes o siglas,
    genera el markdown respentando los saltos de línea y algunos marcadores que identifiques como títulos, subtítulos, negritas o cursivas.

    por ejemplo:

    TEXTO... -> TRADUCCIÓN ... <abbr title="significado original o de las siglas">palabra difícil</abbr>

    preserva las líneas los más fiel posible para que la traducción se vea natural.

    El texto a traducir es al idioma <{target_language}>:

    ===
    {text}
    ===

    NOTA IMPORTANTE: No pongas comentarios adicionales solo el texto traducido.

    """.strip()

    response4_1 = client.responses.create(
        model="gpt-4o",
        input=prompt4_1
    )

    with open(f"{namespace}/reponse4_1.txt", "w") as file:
        file.write(response4_1.output_text)
    
    with open(f"{namespace}/translation.txt", "w") as file:
        file.write(response4_1.output_text)

    prompt5 = f"""

    Genera las 5 ideas principales en el mismo idioma (mantén las 5 ideas en el idioma nativo: <{report1.iloc[0]['language']}>) 
    y que sean breves y resuman el contenido del texto.

    Devuelve una lista JSON con los textos de las 5 ideas para usarlos dentro de python, ejemplo:

    TEXTO... -> [idea1, idea2, idea3, idea4, idea5]

    NOTA IMPORTANTE: Devuelve sólo el JSON sin texto adicional, preámbulos o las tildels ```json
    ya que se usuará directamente en python.

    El texto para extraer las 5 ideas es:

    ===
    {text}
    ===

    """.strip()

    response5 = client.responses.create(
        model="gpt-4o",
        input=prompt5
    )

    with open(f"{namespace}/reponse5.txt", "w") as file:
        file.write(response5.output_text)

    ideas = "* " + "\n* ".join(json.loads(re.sub(r"\`{3}(json)?", "", response5.output_text)))

    with open(f"{namespace}/ideas_native.txt", "w") as file:
        file.write(ideas)

    prompt6 = f"""

    Traduce fielmente las ideas al idioma de traducción <{target_language}>.

    Devuelve una lista JSON con los textos de las 5 ideas traducidas para usarlos dentro de python, ejemplo:

    TEXTO... -> [idea1, idea2, idea3, idea4, idea5]

    NOTA IMPORTANTE: Devuelve sólo el JSON sin texto adicional, preámbulos o las tildels ```json
    ya que se usuará directamente en python.

    El texto las ideas a traducir es:

    ===
    {ideas}
    ===

    """.strip()

    response6 = client.responses.create(
        model="gpt-4o",
        input=prompt6
    )

    with open(f"{namespace}/reponse6.txt", "w") as file:
        file.write(response6.output_text)

    ideas_translate = "* " + "\n* ".join(json.loads(re.sub(r"\`{3}(json)?", "", response6.output_text)))

    with open(f"{namespace}/ideas_translated.txt", "w") as file:
        file.write(ideas_translate)

In [2]:
from openai import OpenAI

with open("openai.key", "r") as file:
    api_key = file.read().strip()

client = OpenAI(api_key=api_key)

print(client.base_url)

https://api.openai.com/v1/


In [10]:
text = """

Hola, me da gusto conocerte, espero que todo lo que hemos platicado por chat sea verdadero.

Viajaré para verte y transferiré la cantidad que me pediste por correo para que me recojas en el
aeropuerto.

Nos vemos pronto.

Te amo, querida amante francesa por correo.

"""

namespace = "s001d"
target_language = "french"

traductor_pro(client, text, namespace, target_language)

In [ ]:
text = """

Hola, me da gusto conocerte, espero que todo lo que hemos platicado por chat sea verdadero.

Viajaré para verte y transferiré la cantidad que me pediste por correo para que me recojas en el
aeropuerto.

Nos vemos pronto.

Te amo, querida amante francesa por correo.

"""

namespace = "s001m"
target_language = "french"

traductor_pro(client, text, namespace, target_language)

In [23]:
text = """

Hola, me da gusto conocerte, espero que todo lo que hemos platicado por chat sea verdadero.

Viajaré para verte y transferiré la cantidad que me pediste por correo para que me recojas en el
aeropuerto.

Nos vemos pronto.

Te amo, querida amante francesa por correo.

"""

namespace = "s001n"
target_language = "alemán"

traductor_pro(client, text, namespace, target_language)